# Analyse Exploratoire Détaillée (Deep EDA) - WILDS CAMELYON17

Ce notebook présente une synthèse complète de l'analyse exploratoire effectuée sur le dataset **CAMELYON17** (version WILDS). L'objectif est de quantifier les défis du dataset (volume, domain shift) et de valider la qualité des données avant la phase de modélisation.

## 📌 Résumé Exécutif

| Catégorie | Résultat | Observation |
|---|---|---|
| **Volume** | 455,954 patchs | Dataset massif et complet. |
| **Structure** | 43 patients / 5 hôpitaux | Répartition géographique claire (1 hôpital par patient). |
| **Équilibre** | **50% / 50%** | Dataset parfaitement équilibré entre Normal et Tumeur par hôpital. |
| **Domain Shift** | **Significatif** | Différences de luminosité et de balance des couleurs entre hôpitaux (Center 4 vs Center 1). |
| **Qualité** | Excellente | Très peu de patchs flous ou vides (<0.32%). |

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from PIL import Image
from tqdm.auto import tqdm
import cv2  # Pour l'analyse de qualité

# Chemins
ROOT_DIR = Path('../data/raw/wilds/camelyon17_v1.0')
PATCHES_DIR = ROOT_DIR / 'patches'
METADATA_PATH = ROOT_DIR / 'metadata.csv'

print(f"Initialisation : {ROOT_DIR}")

## 1. Chargement et Inventaire

Nous chargeons les métadonnées et vérifions la cohérence globale ainsi que la répartition des splits WILDS.

In [ ]:
df = pd.read_csv(METADATA_PATH, index_col=0)
df['Label'] = df['tumor'].map({0: 'Normal', 1: 'Tumeur'})
df['Center_Name'] = 'Hôpital ' + df['center'].astype(str)
df['Split_Name'] = df['split'].map({0: 'Train/Val (ID)', 1: 'Test (OOD)'})

print(f"Nombre total de patchs: {len(df):,}")
print(f"Nombre de patients: {df['patient'].nunique()}")
print(f"Nombre d'hôpitaux: {df['center'].nunique()}")

# Distribution des Splits
split_counts = df['Split_Name'].value_counts().reset_index()
split_counts.columns = ['Split', 'Nombre']
fig_split = px.pie(split_counts, values='Nombre', names='Split', 
                   title="Répartition des Splits WILDS",
                   color_discrete_sequence=px.colors.qualitative.Pastel)
fig_split.show()

df.head()

## 2. Analyse par Hôpital & Domain Shift

Le Domain Shift est un défi majeur dans ce dataset : chaque hôpital possède ses propres scanners et protocoles de coloration.

In [ ]:
# Distribution par hôpital et équilibre des classes
hosp_counts = df.groupby(['Center_Name', 'Label']).size().reset_index(name='Nombre')

fig = px.bar(hosp_counts, x="Center_Name", y="Nombre", color="Label", 
             title="Équilibre des Classes par Hôpital",
             barmode="group",
             color_discrete_map={'Normal': '#2ca02c', 'Tumeur': '#d62728'})
fig.show()

print("Observation : Chaque hôpital présente un équilibre parfait 50/50 entre classes.")

### Quantification du Domain Shift (Statistiques RGB)

Nous échantillonnons 500 patchs par hôpital pour calculer les statistiques de couleur.

In [ ]:
def get_hosp_rgb_stats(n_samples=500):
    centers = sorted(df['center'].unique())
    hosp_stats = []
    
    for center in tqdm(centers, desc="Analyse RGB par hôpital"):
        sample = df[df['center'] == center].sample(n_samples, random_state=42)
        r_vals, g_vals, b_vals = [], [], []
        
        for _, row in sample.iterrows():
            fname = f"patch_patient_{row['patient']:03d}_node_{row['node']}_x_{row['x_coord']}_y_{row['y_coord']}.png"
            path = PATCHES_DIR / f"patient_{row['patient']:03d}_node_{row['node']}" / fname
            if path.exists():
                img = np.array(Image.open(path).convert('RGB'))
                r_vals.append(img[:,:,0].mean())
                g_vals.append(img[:,:,1].mean())
                b_vals.append(img[:,:,2].mean())
        
        hosp_stats.append({
            'Hôpital': f'Hosp_{center}',
            'Red': np.mean(r_vals),
            'Green': np.mean(g_vals),
            'Blue': np.mean(b_vals),
            'Luma': 0.299*np.mean(r_vals) + 0.587*np.mean(g_vals) + 0.114*np.mean(b_vals)
        })
    return pd.DataFrame(hosp_stats)

rgb_df = get_hosp_rgb_stats()
fig_shift = px.line(rgb_df, x='Hôpital', y=['Red', 'Green', 'Blue', 'Luma'], 
                    title="Variation de la Colorimétrie par Hôpital (Domain Shift)",
                    markers=True,
                    labels={'value': 'Intensité Moyenne (0-255)', 'variable': 'Canal'})
fig_shift.show()

## 3. Analyse Niveau Patient

Vérifions le volume de données par patient et l'hétérogénéité de la charge tumorale.

In [ ]:
patient_stats = df.groupby('patient').agg({
    'tumor': ['count', 'mean'],
    'node': 'nunique'
})
patient_stats.columns = ['n_patches', 'tumor_fraction', 'n_nodes']
patient_stats = patient_stats.reset_index()

fig_patch_count = px.histogram(patient_stats, x="n_patches", nbins=15,
                               title="Nombre de Patchs par Patient",
                               labels={'n_patches': 'Volume de Patchs'}, color_discrete_sequence=['#1f77b4'])
fig_patch_count.show()

fig_tf = px.histogram(patient_stats[patient_stats['tumor_fraction'] > 0], x="tumor_fraction", nbins=20,
                      title="Fraction Tumorale par Patient",
                      labels={'tumor_fraction': 'Ratio T / Total'}, color_discrete_sequence=['#ff7f0e'])
fig_tf.show()

print(f"Nodes par patient: Moyenne {patient_stats['n_nodes'].mean():.2f}, Max {patient_stats['n_nodes'].max()}")

## 4. Analyse de Qualité & Flou

Un échantillon de 5000 patchs a été analysé pour détecter le flou (variance du Laplacien) et les fonds blancs vides.

In [ ]:
def analyze_quality_sample(n_samples=1000):
    sample = df.sample(n_samples, random_state=42)
    results = []
    
    for _, row in tqdm(sample.iterrows(), total=n_samples, desc="Qualité"):
        fname = f"patch_patient_{row['patient']:03d}_node_{row['node']}_x_{row['x_coord']}_y_{row['y_coord']}.png"
        path = PATCHES_DIR / f"patient_{row['patient']:03d}_node_{row['node']}" / fname
        if path.exists():
            img = cv2.imread(str(path))
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            blur_var = cv2.Laplacian(gray, cv2.CV_64F).var()
            mean_val = np.mean(img)
            std_val = np.std(img)
            results.append({'blur_var': blur_var, 'mean': mean_val, 'std': std_val})
            
    return pd.DataFrame(results)

quality_df = analyze_quality_sample()

fig_quality = px.histogram(quality_df, x="blur_var", title="Distribution du Score de Netteté (Variance du Laplacien)",
                          log_y=True, labels={'blur_var': 'Score de Netteté'})
fig_quality.add_vline(x=100, line_dash="dash", line_color="red", annotation_text="Seuil Flou")
fig_quality.show()

print(f"Pourcentage de patchs potentiellement flous (<100) : {(quality_df['blur_var'] < 100).mean()*100:.2f}%")

## 5. Visualisation Qualitative

Affichons des patchs normaux et tumoraux pour comparaison visuelle directe.

In [ ]:
def plot_patch_grid(label=0, n=8):
    sample = df[df['tumor'] == label].sample(n, random_state=42)
    plt.figure(figsize=(16, 4))
    
    for i, (_, row) in enumerate(sample.iterrows()):
        fname = f"patch_patient_{row['patient']:03d}_node_{row['node']}_x_{row['x_coord']}_y_{row['y_coord']}.png"
        path = PATCHES_DIR / f"patient_{row['patient']:03d}_node_{row['node']}" / fname
        img = Image.open(path)
        
        plt.subplot(1, n, i+1)
        plt.imshow(img)
        plt.title(f"Hosp {row['center']}")
        plt.axis('off')
    
    plt.suptitle(f"Exemples de patchs {'Normaux' if label==0 else 'Tumoraux'}", fontsize=16)
    plt.tight_layout()
    plt.show()

plot_patch_grid(label=0) # Normal
plot_patch_grid(label=1) # Tumeur